# Monte Carlo Methods: Coursework

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

## Efficiency of Monte Carlo sampling

The calculation will repeat the setup from lab 2, using a Lennard-Jones fluid, briefly repeated below. However, only a single set of physical parameters will be considered.

Use a Lennard-Jones potential inside a box size $[0,L]^3$ with a cut-off $r_c = L/2$, 

$$
\begin{equation}
U = \begin{cases} 4 \left[ \frac{1}{r^{12}} - \frac{1}{r^6} \right] & r < r_c \\ 0 & r > r_c. \end{cases}
\end{equation}
$$

Include tail corrections (that is, additional energy and pressure terms resulting from the particles outside the cutoff radius) as

$$
\begin{align}
  U^{\text{tail}} & = \frac{8 \pi \rho}{3} \left[ \frac{1}{3} \frac{1}{r_c^9} - \frac{1}{r_c^3} \right] \\
  p^{\text{tail}} & = \frac{16 \pi \rho^2}{3} \left[ \frac{2}{3} \frac{1}{r_c^9} - \frac{1}{r_c^3} \right].
\end{align}
$$

For each configuration we need to compute the pressure using

$$
\begin{equation}
  p = \frac{\rho}{\beta} + \frac{\text{Virial}}{3 V}
\end{equation}
$$

where

$$
\begin{equation}
  \text{Virial} = \sum_i \sum_{j > i} \vec{f}( \vec{r}_{ij} ) \cdot \vec{r}_{ij}
\end{equation}
$$

where, as usual, $\vec{r}_{ij}$ is the separation between the atoms, $\vec{r}_{ij} = \vec{r}_i - \vec{r}_j$, and the intermolecular force $\vec{f}$ is given by

$$
\begin{align}
  \vec{f}(\vec{r}_{ij}) &= - \nabla U \\
  & = \begin{cases} 24 \left[ 2 \frac{1}{r^{14}} - \frac{1}{r^8} \right] \vec{r}_{ij} & r < r_c \\ \vec{0} & r > r_c \end{cases}
\end{align}
$$

Note that in the reduced coordinates $\beta = T^{-1}$.

We will be using an $NTV$ approach, keeping the number of particles fixed ($N = 50$), the temperature fixed at $T=2$ and the volume fixed (indirectly, via the density $\rho = N / V = N L^{-3}$, using $\rho = 1/2$). Take $5,000$ steps.

## Tasks

Compare the efficiency of the standard Monte Carlo algorithm, as used in lab 2, with

1. an algorithm where each trial solution moves *all* particles by a random amount, and
2. an algorithm where each trial solution moves a single particle, but by an amount drawn from a Gaussian distribution instead of a uniform distribution.

In all three cases, run the algorithm three times, and plot the average of

$$
\begin{equation}
  \log \left| E - \min (E) + 1 \right|
\end{equation}
$$

against the number of iterations to show the relative efficiency of the algorithms. The size of the random move should be roughly tuned in each case so that the acceptance rate is $\sim 50\%$ - this will require different values of $\Delta$ in each case, and I would recommend starting with $\Delta \sim 0.1$ in the standard case and reducing by roughly a factor of $4$ for the Gaussian case, and roughly a further factor of $4$ when moving all particles.

In [2]:
%matplotlib inline
import numpy
from scipy import constants
from matplotlib import pyplot
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12,6)

from numba import jit